# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import warnings
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
engine = create_engine('sqlite:///DisasterResponse.db')
engine.table_names()

['InsertTableName', 'Message', 'Messages']

In [3]:
# load data from database
df = pd.read_sql_table('Messages', con=engine)
X = df['message']
Y = df.drop(['id','message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    '''
    a tokenizer function that return a tokenized version of a text
    
    input:
    text:   a string conytaining the text to tokenize
    
    output:
    tokens: a list of token representing the input
    
    '''
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens
    

In [5]:
tokenize(X[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
clf = MultiOutputClassifier(RandomForestClassifier())

pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', clf)
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test= train_test_split(X,Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred=pipeline.predict(X_test)


In [9]:
print(classification_report(y_test, y_pred, target_names=y_test.columns.tolist()))

                        precision    recall  f1-score   support

               related       0.85      0.92      0.88      4981
               request       0.82      0.43      0.56      1146
                 offer       0.00      0.00      0.00        31
           aid_related       0.76      0.58      0.66      2763
          medical_help       0.64      0.14      0.23       538
      medical_products       0.74      0.09      0.16       317
     search_and_rescue       0.62      0.07      0.12       189
              security       0.00      0.00      0.00       119
              military       0.58      0.12      0.20       213
           child_alone       0.00      0.00      0.00         0
                 water       0.81      0.39      0.53       427
                  food       0.86      0.53      0.65       752
               shelter       0.85      0.28      0.42       649
              clothing       0.67      0.05      0.09       123
                 money       0.50      

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f61c4d83840>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [11]:
parameters = {'tfidf__norm': ['l2', 'l1'],
             'vect__ngram_range': [(1, 1),(1,2)],
             'clf__estimator__max_depth': [None,2,4],
             'clf__estimator__max_features': ['sqrt', 'log2']}


cv = GridSearchCV(pipeline, parameters, scoring='f1_weighted')
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__norm': ['l2', 'l1'], 'vect__ngram_range': [(1, 1), (1, 2)], 'clf__estimator__max_depth': [None, 2, 4], 'clf__estimator__max_features': ['sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
new_pred=cv.predict(X_test)
print(classification_report(y_test, new_pred, target_names=y_test.columns.tolist()))

                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      4981
               request       0.82      0.46      0.59      1146
                 offer       0.00      0.00      0.00        31
           aid_related       0.78      0.59      0.67      2763
          medical_help       0.60      0.11      0.19       538
      medical_products       0.80      0.10      0.18       317
     search_and_rescue       0.65      0.06      0.11       189
              security       1.00      0.01      0.02       119
              military       0.68      0.07      0.13       213
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.31      0.45       427
                  food       0.84      0.49      0.62       752
               shelter       0.88      0.26      0.40       649
              clothing       0.67      0.07      0.12       123
                 money       0.50      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
clfb=DecisionTreeClassifier()

pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', clfb)
                    ])
params={'tfidf__norm': ['l2', 'l1'],
             'vect__ngram_range': [(1, 1),(1,2)],
             'clf__max_depth': [None,2,4],
             'clf__max_features': [None,'sqrt']
    
}
cvb = GridSearchCV(pipeline, params, scoring='f1_weighted')
cvb.fit(X_train, y_train)
newer_pred=cvb.predict(X_test)
print(classification_report(y_test, newer_pred, target_names=y_test.columns.tolist()))

### 9. Export your model as a pickle file

In [ ]:

pickle.dump(cv, open('myfiles.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.